In [1]:
from go.keras.GoNNet import GoNNet as GoNNet5x5
from go.GoGame import GoGame
from utils import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

2022-11-08 01:14:17.317050: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
args = dotdict({
  'lr': 0.01,
  'dropout': 0.3,
  'epochs': 10,
  'batch_size': 256,
  'cuda': True,
  'num_channels': 512,
})

In [3]:
game = GoGame(n=5)
nnet5x5 = GoNNet5x5(game, args)
nnet5x5.model.summary()

2022-11-08 01:14:18.241407: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-11-08 01:14:18.294405: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-08 01:14:18.294884: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.835GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2022-11-08 01:14:18.294913: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-11-08 01:14:18.297077: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-11-08 01:14:18.299033: I tensorflow/stream_executor/platfo

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 5, 5)]       0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 5, 5, 1)      0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 5, 5, 512)    5120        reshape[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 5, 5, 512)    2048        conv2d[0][0]                     
_______________________________________________________________________________________

In [4]:
nnet5x5.model.get_weights()

[array([[[[ 0.03137908, -0.01956799,  0.02230266, ...,  0.01764959,
            0.02580802, -0.03132408]],
 
         [[ 0.0024012 ,  0.01507997, -0.0188094 , ...,  0.01294006,
            0.0055863 ,  0.01586689]],
 
         [[-0.0205828 , -0.01536184, -0.01725623, ..., -0.01270795,
            0.00799539,  0.03175547]]],
 
 
        [[[-0.03600707, -0.03121741, -0.02267965, ...,  0.01279394,
            0.02313134, -0.00175416]],
 
         [[ 0.02672595,  0.00498415,  0.01591305, ..., -0.00700219,
            0.00013116,  0.0136941 ]],
 
         [[ 0.01021755, -0.02358978,  0.01673877, ..., -0.02216285,
           -0.01293785, -0.01166314]]],
 
 
        [[[-0.01285948,  0.03512156,  0.03500772, ..., -0.03227124,
            0.01016434,  0.01510183]],
 
         [[ 0.00558162, -0.03204595, -0.0189705 , ...,  0.03153857,
            0.00760231,  0.03423293]],
 
         [[ 0.03265968,  0.03392664, -0.01941026, ..., -0.03313648,
            0.01132968, -0.02852937]]]], dtype=float32

In [5]:
nnet5x5.model.load_weights('./temp (5x5)/best.h5')

In [6]:
nnet5x5.model.get_weights()

[array([[[[ -0.15716667,  -2.5174243 ,   0.35057425, ...,   0.0422143 ,
            -1.6608897 ,  -1.5470461 ]],
 
         [[ -0.03845736,  -1.6797414 ,  -2.0988789 , ...,   0.11925497,
            -2.977605  ,   3.932173  ]],
 
         [[ -4.8674774 ,   2.3281772 ,   0.23044094, ...,   0.09136806,
            -2.2898867 ,  -0.63536024]]],
 
 
        [[[  1.870998  ,  -0.8592456 ,  -3.6898162 , ...,   0.2003669 ,
            -2.1412868 ,  -1.6146194 ]],
 
         [[  0.42363143,   0.5388574 ,   2.5201757 , ..., -33.175716  ,
             3.0035162 ,   1.6791655 ]],
 
         [[ -0.87232596,  -1.6891435 ,  -4.214603  , ...,  -0.17217511,
            -0.5447151 ,  -1.3394876 ]]],
 
 
        [[[ -4.4849434 ,   3.7789707 ,   0.38153732, ...,   0.24555963,
            -3.0172024 ,  -3.4485245 ]],
 
         [[  0.36316058,  -0.5684589 ,  -1.915556  , ...,   0.24471858,
             0.37651423,   2.564024  ]],
 
         [[  1.71666   ,  -3.2343318 ,  -0.22930208, ...,   0.11666921,
  

In [7]:
nnet5x5.model.layers

In [8]:
nnet5x5.model.trainable = False

inputs = Input(shape=(7, 7))
x = Reshape((7, 7, 1))(inputs)
old_layers = nnet5x5.model.layers[2:-10]
for l in old_layers:
    x = l(x)
x = Dropout(args.dropout)(Activation('relu')(BatchNormalization(axis=1)(Dense(1024)(x)))) # batch_size x 1024
x = Dropout(args.dropout)(Activation('relu')(BatchNormalization(axis=1)(Dense(512)(x))))  # batch_size x 512
pi = Dense(7 * 7 + 1, activation='softmax', name='pi')(x) # batch_size x self.action_size
v = Dense(1, activation='tanh', name='v')(x)              # batch_size x 1
model = Model(inputs=inputs, outputs=[pi, v])
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 7, 7)]       0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 7, 7, 1)      0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 multiple             5120        reshape_1[0][0]                  
__________________________________________________________________________________________________
batch_normalization (BatchNorma multiple             2048        conv2d[1][0]                     
_______________________________________________________________________________________

In [11]:
model.get_weights()

[array([[[[ -0.15716667,  -2.5174243 ,   0.35057425, ...,   0.0422143 ,
            -1.6608897 ,  -1.5470461 ]],
 
         [[ -0.03845736,  -1.6797414 ,  -2.0988789 , ...,   0.11925497,
            -2.977605  ,   3.932173  ]],
 
         [[ -4.8674774 ,   2.3281772 ,   0.23044094, ...,   0.09136806,
            -2.2898867 ,  -0.63536024]]],
 
 
        [[[  1.870998  ,  -0.8592456 ,  -3.6898162 , ...,   0.2003669 ,
            -2.1412868 ,  -1.6146194 ]],
 
         [[  0.42363143,   0.5388574 ,   2.5201757 , ..., -33.175716  ,
             3.0035162 ,   1.6791655 ]],
 
         [[ -0.87232596,  -1.6891435 ,  -4.214603  , ...,  -0.17217511,
            -0.5447151 ,  -1.3394876 ]]],
 
 
        [[[ -4.4849434 ,   3.7789707 ,   0.38153732, ...,   0.24555963,
            -3.0172024 ,  -3.4485245 ]],
 
         [[  0.36316058,  -0.5684589 ,  -1.915556  , ...,   0.24471858,
             0.37651423,   2.564024  ]],
 
         [[  1.71666   ,  -3.2343318 ,  -0.22930208, ...,   0.11666921,
  

In [12]:
model.save_weights('temp/best.h5')